<a href="https://colab.research.google.com/github/vjumpkung/vjump-sd-webui-colab/blob/main/vjumpkung_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ComfyUI colab (created by vjumpkung)**

## แจ้งปัญหาการใช้งาน

https://discord.com/users/250567674504019968

discord username : vjump

### สิ่งที่ต้องการ

- ต้องใช้ Colab Pro หรือ Paid Service เท่านั้น Colab Free ไม่สามารถใช้งานได้

### คู่มือการใช้งาน

- https://vjump-sd-download-list-docs.vercel.app/comfy

### Website แหล่งรวม SD Model List เพื่อสำหรับการใช้งาน Colab นี้

https://vjump-sd-download-list.vjumpkung.dynv6.net/download_list/

### Features

- Save รูป ไว้ที่ Google Drive ทุกรูปที่ได้เจนภาพขึ้นมา
- มี ComfyUI Manager ในการจัดการ Custom Node เพื่อความสะดวกในการติดตั้ง custom node ที่สะดวกสะบายมากขึ้น
- รองรับการ download model จาก civitai รวมถึง model ที่ต้องการ login และ huggingface
- มีเว็บไซต์สำหรับแหล่งรวม model ต่างๆ เพื่อจะได้ download model ทั้งหมดภายใน url เดียว

## Changelog

### Update 14 August 2024

- รองรับ Flux NF4

### Update 11 August 2024

- รองรับ Flux, Flux FP8 โดยเปลี่ยน runtime type เป็น L4 เพื่อใช้งาน Flux FP8

### Reference

- modification Python Notebook from https://github.com/comfyanonymous/ComfyUI/blob/master/notebooks/comfyui_colab.ipynb



In [1]:
#@title # Installing ComfyUI (ติดตั้ง ComfyUI)
from pprint import pp
print("Init Notebook Completed")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from pathlib import Path
from IPython.display import clear_output
import ipywidgets as widgets
import sys
import os
import requests
MODEL_PATH = "/content/comfyui_models"
!mkdir -p /content/comfyui_models/models/{checkpoints,clip,clip_vision,configs,controlnet,diffusers,diffusers_models,embeddings,gligen,hypernetworks,ipadapter,loras,photomaker,style_models,unet,upscale_models,vae,vae_approx}
if 'local_storage' in globals():
    pass
else:
    local_storage = []
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb &> /dev/null
!dpkg -i cloudflared-linux-amd64.deb &> /dev/null
!apt-get -y install -qq aria2 &> /dev/null
if not os.path.exists(f"/content/venv"):
    !apt install python3.10-venv &> /dev/null
    !python -m venv /content/venv
clear_output()

!mkdir -p /content/Downloads

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

from pathlib import Path

OPTIONS = {}

#@markdown ### Save Model ไว้ที่ Google Drive
SAVE_MODEL_IN_GOOGLE_DRIVE = False  #@param {type:"boolean"}
#@markdown ### Update ComfyUI ถ้าติ๊ก SAVE_MODEL_IN_GOOGLE_DRIVE ถูกเอาไว้
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = SAVE_MODEL_IN_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    %cd /
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive
    print("Copying Models into RUNTIME")
    !cp -r {WORKSPACE}/models/ {MODEL_PATH}

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone --quiet https://github.com/comfyanonymous/ComfyUI > /dev/null
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
    !echo -= Updating ComfyUI =-
    !git reset --hard
    !git pull

!echo -= Install dependencies =-
!/content/venv/bin/pip install -q ipykernel > /dev/null
!/content/venv/bin/pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121 > /dev/null
!/content/venv/bin/pip install -r requirements.txt > /dev/null
%cd $WORKSPACE
%cd custom_nodes
!echo -= Install ComfyUI Manager =-
!git clone --quiet https://github.com/ltdrdata/ComfyUI-Manager.git > /dev/null
!git clone --quiet https://github.com/comfyanonymous/ComfyUI_bitsandbytes_NF4.git > /dev/null
!/content/venv/bin/pip install bitsandbytes > /dev/null
!/content/venv/bin/pip install -r ./ComfyUI-Manager/requirements.txt > /dev/null
!/content/venv/bin/python {WORKSPACE}/custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies > /dev/null
%cd $WORKSPACE

def download(Type:str,link:str):
    URL = link
    if not os.path.exists(f"{MODEL_PATH}/models/{Type}/"):
        !mkdir -p {MODEL_PATH}/models/{Type}
    print(f"Downloading {link}")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --dir=/content/Downloads/ --content-disposition=true {URL}
    exit_code = get_ipython().__dict__['user_ns']['_exit_code']
    if exit_code != 0:
        print(f"Cannot Download {link}")
        return False
    !mv /content/Downloads/* {MODEL_PATH}/models/{Type}
    print(f"Download {Type} from {link} completed")
    return True

# create models location yaml config
model_location = '''# ComfyUI models location
comfyui:
    checkpoints: /content/comfyui_models/models/checkpoints/
    clip: /content/comfyui_models/models/clip/
    clip_vision: /content/comfyui_models/models/clip_vision/
    configs: /content/comfyui_models/models/configs/
    controlnet: /content/comfyui_models/models/controlnet/
    diffusers: /content/comfyui_models/models/diffusers/
    diffusers_models: /content/comfyui_models/models/diffusers_models/
    embeddings: /content/comfyui_models/models/embeddings/
    gligen: /content/comfyui_models/models/gligen/
    hypernetworks: /content/comfyui_models/models/hypernetworks/
    ipadapter: /content/comfyui_models/models/ipadadapter/
    loras: /content/comfyui_models/models/loras/
    photomaker: /content/comfyui_models/models/photomaker/
    style_models: /content/comfyui_models/models/style_models/
    unet: /content/comfyui_models/models/unet/
    upscale_models: /content/comfyui_models/models/upscale_models/
    vae: /content/comfyui_models/models/vae/
    vae_approx: /content/comfyui_models/models/vae_approx/
'''
print("Creaete Extra Model Paths Config")
!touch {WORKSPACE}/extra_model_paths.yaml
with open(f"{WORKSPACE}/extra_model_paths.yaml", "w") as fp:
    fp.write(model_location)

comfyui_map = {
    'Stable-diffusion':'checkpoints',
    'ControlNet':'controlnet',
    'embeddings':'embeddings',
    'hypernetworks':'hypernetworks',
    'VAE':'vae',
    'Lora':'loras',
    'ESRGAN':'upscale_models'
}

# @markdown ### Download Model from Link รวมรายการ Download
# @markdown สามารถค้นหาได้จาก website นี้ https://vjump-sd-download-list.vjumpkung.dynv6.net/download_list/ หรือ Link ที่เก็บไว้จากการสร้าง URL ในขั้นตอน Generate Download List URL
# @markdown <br>คู่มือการใช้งาน SD Download List https://vjump-sd-download-list-docs.vercel.app/sd-dl-list
DOWNLOAD_LIST_URL = "" #@param {type:"string"}
# @markdown ### CivitAI API Keys (สำหรับ Download Model ที่ต้องการ Login)
# @markdown วิธีการขอ CivitAI API Key https://vjump-sd-download-list-docs.vercel.app/others/civit
CIVITAI_API_KEYS = "" #@param {type: "string"}
if DOWNLOAD_LIST_URL != "":
    r = requests.get(DOWNLOAD_LIST_URL).json()
    if "download_list" not in r.keys() or r["ui_type"] not in ["comfy","A1111"]:
        print("Invalid URL")
    else:
        for i in r["download_list"]:
            if r["ui_type"] == "A1111":
                if i["DownloadModelType"] in comfyui_map.keys():
                    i["DownloadModelType"] = comfyui_map[i["DownloadModelType"]]
            if CIVITAI_API_KEYS != "" and i["source"] == "CivitAI":
                download(Type=i["DownloadModelType"],link=i["downloadUrl"]+f"?token={CIVITAI_API_KEYS}")
            else:
                download(Type=i["DownloadModelType"],link=i["downloadUrl"])
clear_output()
inf('\u2714 Install ComfyUI Completed','success', '200px')

Button(button_style='success', description='✔ Install ComfyUI Completed', disabled=True, layout=Layout(min_wid…

In [ ]:
# @title # Download Model (โหลด Model ใหม่)
# @markdown ## รองรับ model จาก civitai, huggingface
#@markdown - [วิธีการ Download Model จากเว็บ CivitAI](https://vjump-sd-download-list-docs.vercel.app/comfy#21-%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5-download-%E0%B8%88%E0%B8%B2%E0%B8%81%E0%B9%80%E0%B8%A7%E0%B9%87%E0%B8%9A-civitai)
#@markdown - [วิธีการ Download Model จากเว็บ Huggingface](https://vjump-sd-download-list-docs.vercel.app/comfy#22-%E0%B8%A7%E0%B8%B4%E0%B8%98%E0%B8%B5-download-%E0%B8%88%E0%B8%B2%E0%B8%81%E0%B9%80%E0%B8%A7%E0%B9%87%E0%B8%9A-huggingface)

# @markdown #### ทุกๆ การ download จะมีการเก็บประวัติการ download เอาไว้ใน colab ซึ่งถ้าต้องการเก็บประวัติเอาไว้สามารถกด run Generate Download List URL เพื่อสร้าง link ที่เก็บประวัติการ download ไว้ได้
import requests
from IPython import get_ipython
from urllib.parse import urlparse
from pprint import pp
import os
def download(Type:str,link:str):
    URL = link
    if not os.path.exists(f"{MODEL_PATH}/models/{Type}/"):
        !mkdir -p {MODEL_PATH}/models/{Type}
    print(f"Downloading {link}")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --dir=/content/Downloads/ --content-disposition=true {URL}
    exit_code = get_ipython().__dict__['user_ns']['_exit_code']
    if exit_code != 0:
        print(f"Cannot Download {link}")
        return False
    !mv /content/Downloads/* {MODEL_PATH}/models/{Type}
    print(f"Download {Type} from {get_down['downloadUrl']} completed")
    return True

def get_url_link(url):
    if "civitai" in url:
        parsedUrl = urlparse(url)
        modelId = int(parsedUrl.path.split("/")[2])
        versionId = None
        if parsedUrl.query != "":
            versionId = int(parsedUrl.query.split("=")[-1])
        getUrl = requests.get(f"https://civitai.com/api/v1/models/{modelId}")
        modelJson = {}
        if getUrl.status_code == 200:
            modelJson = getUrl.json()
        else:
            print("Invalid CivitAI URL")
            return
        getModelInfo = modelJson["modelVersions"][0]
        for i in modelJson["modelVersions"]:
            if i["id"] == versionId:
                getModelInfo = i
                break
        fn = ""
        durl = ""
        sortedImagePreview = getModelInfo["images"]
        previewImage = sortedImagePreview[0]

        for i in getModelInfo["files"]:
            if "primary" in i.keys():
                if i["primary"] == True:
                    fn = i["name"]
                    durl = i["downloadUrl"]
                    break
        res = {
            "model_id": modelJson["id"],
            "source": "CivitAI",
            "type": modelJson["type"],
            "baseModel": getModelInfo["baseModel"],
            "name": modelJson["name"],
            "version": getModelInfo["name"],
            "modellink": url,
            "fileName": fn,
            "downloadUrl": durl,
            "previewImage": previewImage,
        }
        return res
    elif "huggingface" in url:
        parsedUrl = urlparse(url)
        durl = url.replace("blob", "resolve") + "?download=true"
        res = {
            "model_id": None,
            "source": "huggingface",
            "type": None,
            "baseModel": None,
            "name": parsedUrl.path.split("/")[-1],
            "version": None,
            "modellink": url,
            "fileName": parsedUrl.path.split("/")[-1],
            "downloadUrl": durl,
            "previewImage": None,
        }
        return res
    else:
        print("Invalid URL")
Type = "checkpoints" #@param ['checkpoints', 'clip_vision', 'controlnet', 'embeddings', 'hypernetworks', 'photomaker', 'unet', 'vae','clip', 'configs', 'diffusers', 'gligen', 'loras', 'style_models', 'upscale_models', 'vae_approx', 'ipadapter']
URL = "" #@param {type:"string"}
# @markdown ### CivitAI API Keys (สำหรับ Download Model ที่ต้องการ Login)
CIVITAI_API_KEYS = "" #@param {type: "string"}
#@markdown วิธีการขอ CivitAI API Key https://vjump-sd-download-list-docs.vercel.app/others/civit <br>
#@markdown *โปรดระบุ Type ให้ถูกต้องเพื่อให้ model ที่ download มาอยู่ถูก folder ตามประเภทของ model ที่ได้กำหนดไว้
get_down = get_url_link(URL)
if (get_down == "Invalid URL"):
    raise
get_down["DownloadModelType"] = Type

if CIVITAI_API_KEYS != "" and get_down["source"] == "CivitAI":
    if download(Type, get_down["downloadUrl"]+f"?token={CIVITAI_API_KEYS}"):
        if get_down not in local_storage:
            local_storage.append(get_down)
        print(f"Saved {Type} - {URL} to download list")
else:
    if download(Type, get_down["downloadUrl"]):
        if get_down not in local_storage:
            local_storage.append(get_down)
        print(f"Saved {Type} - {URL} to download list")

In [ ]:
from random import randint
#@title # Generate Download List URL (สำหรับการนำมาใช้ครั้งถัดไป)
#@markdown ### สร้าง url ที่เก็บประวัติการ download model ต่างๆ เพื่อเก็บไว้ใช้ในครั้งถัดไป
#@markdown หัวข้อ (ไม่บังคับ)
Title = "" #@param {type : "string"}
#@markdown คำอธิยาย (ไม่บังคับ)
Description = "" #@param {type : "string"}
#@markdown ชื่อผู้ใช้ (ไม่บังคับ)
Author = "" #@param {type : "string"}
#@markdown ตั้งค่าการมองเห็น (ติ๊กถูก ✅ หมายถึงเห็นทุกคน ถ้าไม่ได้ติ๊กถูกจะเห็นเฉพาะคนที่มีลิ้งเท่านั้น)
isPublic = False #@param {type : "boolean"}
#@markdown กด run เพื่อสร้าง link ที่รวมรายการ download เพื่อเก็บไว้ใช้ในครั้งถัดไป

if Title == "":
  Title = f"Share Download List {hex(randint(10000,1000000000))[2:]}"
if Description == "":
  Description = f"-"
if Author == "":
  Author = "anonymous"

print("----- Recent Download List -----")
for i in local_storage:
    print(f'Model : {i["name"]} Type : {i["DownloadModelType"]}')
print("--------------------------------")

requrl = "https://sd-download-list-api.vjumpkung.dynv6.net/api/create_download_list_link"
if len(local_storage) > 0:
    req = requests.post(requrl,json={
    "title": Title,
    "description": Description,
    "author": Author,
    "ui_type": "comfy",
    "downloaded_list": local_storage,
    "isPublic": isPublic
    })
    print(f"Keep this url for download every model later.")
    print(f"URL : {req.json()['url']}")
else:
    print("Recent Download Model list not found, Please Download some model")

In [ ]:
#@title # Start ComfyUI
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    time.sleep(5)
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
            #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

%cd $WORKSPACE
!mkdir -p /content/temp/
!/content/venv/bin/python main.py --dont-print-server --output-directory /content/drive/MyDrive/ComfyUIOutput/ --highvram --temp-directory /content/temp/
clear_output()
print("Stop ComfyUI")